In [8]:
%load_ext cython
import matplotlib.pyplot as plt
import meshpy.tet
import numpy as np

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [2]:
import logging


from LoopStructural.cython.dsi_helper import cg
from numpy import linalg as la
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA

logger = logging.getLogger(__name__)


class TetMesh:
    """
    A class containing the geometry of a tetrahedral mesh.
    Nodes are the vertices
    Elements are the tetrahedrons
    Neighbours are the neighbours
    Vertex properties are stored as a dict of np arrrays self.properties
    Element properties are self.property_gradients 
    """

    def __init__(self, **kwargs):
        """
        Creates a mesh object. This just assigns the mesh a name and a
        location to save

        """
        self.name = 'TetMesh'
        self.path = './'
        self.usetetgen = True
        self.save_mesh = False
        if 'name' in kwargs:
            self.name = kwargs['name']
        if 'path' in kwargs:
            self.path = kwargs['path']
        if 'tetgen' in kwargs:
            self.usetetgen = kwargs['tetgen']
        if 'save' in kwargs:
            self.save_mesh = kwargs['save']
        self.mesh = None
        self.shared_idxs = np.zeros(3, dtype=np.int)
        self.dinfo = {}
        self.cg_calculated = {}
        self.properties = {}
        self.property_gradients = {}
        self.property_gradients_nodes = {}
        self.element_gradients = {}
        self.element_nodes_to_id = {}
        self.regions = {}
        self.points = None
        self.surfaces = {}

    def setup_mesh(self, boundary_points, **kwargs):
        """
        Build a mesh given the boundary points
        Can define the resolution of the mesh using the kwargs
         
        n_tetra: number of tetrahedron
        maxvol: maximum volume for a single tetra - if both are specified
        this one is used.
        """
        self.pca = PCA(n_components=3)
        minx = boundary_points[0, 0]
        miny = boundary_points[0, 1]
        minz = boundary_points[0, 2]
        maxx = boundary_points[1, 0]
        maxy = boundary_points[1, 1]
        maxz = boundary_points[1, 2]
        points = np.array([
            (minx, miny, minz),
            (maxx, miny, minz),
            (maxx, maxy, minz),
            (minx, maxy, minz),
            (minx, miny, maxz),
            (maxx, miny, maxz),
            (maxx, maxy, maxz),
            (minx, maxy, maxz)
        ])
        self.surfaces['top'] = (
            (minx, miny, maxz), (maxx, miny, maxz), (maxx, maxy, maxz),
            (minx, maxy, maxz))
        self.surfaces['bottom'] = (
            (minx, miny, minz), (maxx, miny, minz), (maxx, maxy, minz),
            (minx, maxy, minz))

        self.points = points
        # calculate the 3 principal components to find the local coordinate
        # system
        self.pca.fit(points)
        # project the points into this coordinate system
        newp = self.pca.transform(points)
        n_tetra = 4000  # maxvol=0.001
        if 'n_tetra' in kwargs:
            n_tetra = kwargs['n_tetra']
        # calculate the volume of the bounding box
        maxvol = 0.001
        if 'maxvol' not in kwargs:
            lengthU = maxx - minx
            lengthV = maxy - miny
            lengthW = maxz - minz
            boxVol = lengthU * lengthW * lengthV
            correction_factor = 1.91
            maxvol = correction_factor * boxVol / n_tetra
        if 'maxvol' in kwargs:
            maxvol = kwargs['maxvol']

        facets = [
            [0, 1, 2, 3],
            [4, 5, 6, 7],
            [0, 4, 5, 1],
            [1, 5, 6, 2],
            [2, 6, 7, 3],
            [3, 7, 4, 0]
        ]

        # create the mesh
        info = meshpy.tet.MeshInfo()
        # use the projected points to build the mesh
        info.set_points(newp)
        info.set_facets(facets)
        meshpy_mesh = meshpy.tet.build(info, max_volume=maxvol,
                                       options=meshpy.tet.Options('pqn'))
        self.nodes = self.pca.inverse_transform(np.array(meshpy_mesh.points))
        self.elements = np.array(meshpy_mesh.elements, dtype=np.int64)
        self.neighbours = np.array(meshpy_mesh.neighbors, dtype=np.int64)
        self.n_nodes = len(self.nodes)
        self.n_elements = len(self.elements)
        self.barycentre = np.sum(self.nodes[self.elements][:, :, :],
                                 axis=1) / 4.
        self.tree = cKDTree(self.barycentre)
        self.minx = minx  # np.min(newp[:,0])#minx
        self.miny = miny  # np.min(newp[:,1])#miny
        self.minz = minz  # np.min(newp[:,2])#minz
        self.maxx = maxx  # np.max(newp[:,0])#maxx
        self.maxy = maxy  # np.max(newp[:,1])#maxy
        self.maxz = maxz  # np.max(newp[:,2])#maxz

        self.minpc0 = np.min(newp[:, 0])
        self.maxpc0 = np.max(newp[:, 0])
        self.minpc1 = np.min(newp[:, 1])
        self.maxpc1 = np.max(newp[:, 1])
        self.minpc2 = np.min(newp[:, 2])
        self.maxpc2 = np.max(newp[:, 2])

        self.regions['everywhere'] = np.ones(self.n_nodes).astype(bool)

        # precalculate the gradient matrices for all elements to save time
        # later
        e = np.arange(self.n_elements)
        ps = self.nodes[self.elements[e]]
        m = np.array(
            [[(ps[:, 1, 0] - ps[:, 0, 0]), (ps[:, 1, 1] - ps[:, 0, 1]),
              (ps[:, 1, 2] - ps[:, 0, 2])],
             [(ps[:, 2, 0] - ps[:, 0, 0]), (ps[:, 2, 1] - ps[:, 0, 1]),
              (ps[:, 2, 2] - ps[:, 0, 2])],
             [(ps[:, 3, 0] - ps[:, 0, 0]), (ps[:, 3, 1] - ps[:, 0, 1]),
              (ps[:, 3, 2] - ps[:, 0, 2])]])
        I = np.array(
            [[-1., 1., 0., 0.],
             [-1., 0., 1., 0.],
             [-1., 0., 0., 1.]])
        m = np.swapaxes(m, 0, 2)
        self.element_gradients = la.inv(m)

        self.element_gradients = self.element_gradients.swapaxes(1, 2)
        self.element_gradients = self.element_gradients @ I

    def add_region(self, region, name):
        """
        Add a region mask to the mesh, will also add as a property to the mesh
        :param region: numpy array n_nodes
        :param name: string giving name of the region
        :return:
        """
        self.regions[name] = region
        self.properties['REGION_' + name] = region.astype(float)

    def update_property(self, name, value, save=True):
        """
        Updates or adds new property to the mesh
        :param name: string fi
        :param value:
        :param save:
        :return:
        """
        self.properties[name] = value
        grads = self.get_elements_gradients(np.arange(self.n_elements))
        props = self.properties[name][
            self.elements[np.arange(self.n_elements)]]
        grad = np.einsum('ikj,ij->ik', grads, props)
        self.property_gradients[name] = grad
        if self.save_mesh:
            self.save()

    def transfer_gradient_to_nodes(self, propertyname):
        """
        Copy gradient from elements to nodes
        Averages the gradient for the 4 surrounding tetra for each node
        :param propertyname: name of the property
        :return:
        """
        grad = np.zeros((self.nodes.shape[0], 3))
        for i, e in enumerate(self.elements):
            for n in e:
                grad[n, :] += self.property_gradients[propertyname][i, :]
        grad /= 4
        self.property_gradients_nodes[propertyname] = grad

    def calculate_constant_gradient(self, region):
        self.dinfo = np.zeros(self.n_elements).astype(bool)
        # add cg constraint for all of the
        EG = self.get_elements_gradients(np.arange(self.n_elements))
        region = region.astype('int64')
        idc, c, ncons = cg(EG, self.neighbours, self.elements, self.nodes,
                           region)
        idc = np.array(idc[:ncons, :])
        c = np.array(c[:ncons, :])
        B = np.zeros(c.shape[0])
        return c, idc, B

    def get_constant_gradient(self, w=0.1, region='everywhere', **kwargs):
        return self.calculate_constant_gradient(region, **kwargs)

    def get_elements_gradients(self, e):
        """
        Returns the gradient of the elements using their global index.
        Sped up using numpy
        """
        return self.element_gradients[e]

    def calc_bary_c(self, e, p):
        """
        Calculate the barycentre coordinates for an array of n elements 
        and n points
        """
        points = self.nodes[self.elements[e]]
        npts = len(e)
        vap = p - points[:, 0, :]
        vbp = p - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]

        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        return c

    def elements_for_array(self, array, k=10):
        """
        Get the elements for an array of points
        :param array:
        :param k:
        :return:
        """
        # find the nearest k elements for the arrays
        # reducing k could speed up calculation but migh add errors

        d, ee = self.tree.query(array)

        inside = np.zeros(array.shape[0]).astype(bool)
        inside[:] = True
        # inside = iarray[:, 0] > self.minpc0[None]
        # inside *= iarray[:, 0] < self.maxpc0[None]
        # inside *= iarray[:, 1] > self.minpc1[None]
        # inside *= iarray[:, 1] < self.maxpc1[None]
        # inside *= iarray[:, 2] > self.minpc2[None]
        # inside *= iarray[:, 2] < self.maxpc2[None]

        return ee, inside

    def evaluate_value(self, array, prop, region='everywhere'):
        return self.eval_interpolant(array, prop, region)

    def evaluate_gradient(self, array, prop, region='everywhere'):
        return self.eval_gradient(array, prop, region)

    def eval_interpolant(self, array, prop, k=5, e=None, region='everywhere'):
        """
        Evaluate an interpolant from property on an array of points.
        Uses numpy to speed up calculations but could be expensive for large
        mesh/points
        """
        if e == None:
            e, inside = self.elements_for_array(array, k)
        else:
            inside = np.array(e.shape).astype(bool)
            inside[:] = True

        bc = self.calc_bary_c(e[inside], array[inside])
        prop_int = np.zeros(e.shape)
        nv = np.zeros(self.properties[prop].shape)
        nv[~self.regions[region]] = np.nan
        nv[self.regions[region]] = self.properties[prop][self.regions[region]]
        props = self.properties[prop][self.elements[e[inside]]]
        prop_int[inside] = np.sum((bc.T * props), axis=1)
        prop_int[~inside] = np.nan
        return prop_int

    def element_property_value(self, prop):
        """
        Gets the average property value for all of the elements
        :param prop:
        :return:
        """
        bc = np.zeros(4)
        bc[:] = 0.25
        e = np.arange(self.n_elements)
        prop_int = np.zeros(e.shape)

        props = self.properties[prop][self.elements[e]]
        prop_int = np.sum((bc.T * props), axis=1)
        return prop_int

    def element_property_gradient(self, prop):
        """
        Get the gradient of a property for all elements in the mesh
        :param prop:
        :return:
        """
        e = np.arange(self.n_elements)

        grads = self.element_gradients[e]
        vals = self.properties[prop][self.elements[e]]
        # grads = np.swapaxes(grads,1,2)
        a = np.zeros((self.n_elements, 3))  # array.shape)
        a = (grads * vals[:, None, :]).sum(2) / 4.  # @ vals.T/
        a /= np.sum(a, axis=1)[:, None]
        return a

    def eval_gradient(self, array, prop, k=5, region='everywhere'):
        """
        Evaluate an interpolant from property on an array of points.
        Uses numpy to speed up calculations but could be expensive for large
        mesh/points
        """
        e, inside = self.elements_for_array(array, k)
        # ps = self.nodes[self.elements[e[inside]]]
        # m = np.array(
        #     [[(ps[:, 1, 0] - ps[:, 0, 0]), (ps[:, 1, 1] - ps[:, 0, 1]),
        #     (ps[:, 1, 2] - ps[:, 0, 2])],
        #      [(ps[:, 2, 0] - ps[:, 0, 0]), (ps[:, 2, 1] - ps[:, 0, 1]),
        #      (ps[:, 2, 2] - ps[:, 0, 2])],
        #      [(ps[:, 3, 0] - ps[:, 0, 0]), (ps[:, 3, 1] - ps[:, 0, 1]),
        #      (ps[:, 3, 2] - ps[:, 0, 2])]])
        # I = np.array(
        #     [[-1., 1., 0., 0.],
        #      [-1., 0., 1., 0.],
        #      [-1., 0., 0., 1.]])
        # m = np.swapaxes(m, 0, 2)
        # grads = la.inv(m)
        #
        # grads = grads.swapaxes(1, 2)
        # grads = grads @ I
        grads = self.element_gradients[e]
        nv = np.zeros(self.properties[prop].shape)
        nv[~self.regions[region]] = np.nan
        nv[self.regions[region]] = self.properties[prop][self.regions[region]]
        vals = self.properties[prop][self.elements[e[inside]]]
        a = np.zeros(array.shape)
        a[inside] = (grads * vals[:, None, :]).sum(2) / 4.  # @ vals.T/
        a[~inside, :] = np.nan
        asum = np.zeros(a.shape[0])
        asum[inside] = np.sum(a[inside, :], axis=1)
        inside = ~np.isnan(asum)
        logic = np.zeros(asum.shape).astype(bool)
        logic[:] = False
        logic[inside] = asum[inside] > 0
        a[logic] /= asum[logic, None]
        return a



    def slice(self, propertyname, isovalue, region='everywhere'):
        """
        Create a triangular surface from an isovalue of the scalar field
        Parameters
        ----------
        isovalue - value for creating surface for

        Returns
        -------

        """
        logger.error("function has been removed, please use the modelviewer class")
        return



In [2]:
%%cython -a
import numpy as np
def build_aabb(int nx, int ny, int nz, long [:,:] tetra, double [:,:] nodes, long [:,:] aabb, double [:] step):
    cdef int i, ntetra, j, ix, iy, iz
    cdef double [:,:] bb = np.zeros((2,3))
#     cdef double [:] maxs = np.zeros(3)
    cdef double [:,:] points = np.zeros((4,3))
    ntetra = tetra.shape[0]
    # create a mask for extracting corners from 
    cdef long [:,:] corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    for i in range(ntetra):
        # copy points into tmp array
        for k in range(4):
            for l in range(3):
                points[k,l] = nodes[tetra[i,k],l]
        for l in range(3):
            bb[0,l] = np.min(points[:,l])#,axis=0)
            bb[1,l] = np.max(points[:,l])#,axis=0)
        
        for j in range(8):
            ix = int(bb[corners[0,j],0] // step[0])
            iy = int(bb[corners[1,j],1] // step[1])
            iz = int(bb[corners[2,j],2] // step[2])
            aabb[ix+nx*iy+nx*ny*iz,i] = i
        
    return

In [3]:
# %%cython -a
# def find_tetras(double [:,:] nodes, long [:,:] tetras, long [:,:] aabbindx, double [:,:] pos):
#     cdef npts = pos.shape[0]
#     cdef i,j,k
#     for i in range(npts):
#         vap = pos[i,:] - nodes[] 
#     vap = pos[:,None,:] - points[:, :, 0, :]
#         vbp = pos[:,None,:] - points[:, :, 1, :]
#         # vcp = p - points[:, 2, :]
#         # vdp = p - points[:, 3, :]
#         vab = points[:, :, 1, :] - points[:, :, 0, :]
#         vac = points[:, :, 2, :] - points[:, :, 0, :]
#         vad = points[:, :, 3, :] - points[:, :, 0, :]
#         vbc = points[:, :, 2, :] - points[:, :, 1, :]
#         vbd = points[:, :, 3, :] - points[:, :, 1, :]
        
#         va = np.einsum('ikj, iklj->ikl', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
#         vb = np.einsum('ikj, iklj->ikl', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         vc = np.einsum('ikj, iklj->ikl', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
#         vd = np.einsum('ikj, iklj->ikl', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.
#         v = np.einsum('ikj, iklj->ikl', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
#         c = np.zeros((va.shape[0],va.shape[1],4))

In [78]:
class TetMesh2:
    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=2,ny=2,nz=2):
        self.origin = np.array(origin)
        self.maximum = np.array(maximum)
        self.nodes = nodes
        self.tetra = tetra
        self.ntetra = tetra.shape[0]

        self.neighbours = neighbours
        self.step = (self.maximum-self.origin)/(np.array([nx,ny,nz])-1)
        self.aabb = np.zeros((nx*ny*nz,self.ntetra)).astype(int)
        self.aabb[:] = -1
        self.nx = nx
        self.nz = nz
        self.ny = ny
        self._build_aabb()
        self.corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                                      [0, 0, 1, 0, 0, 1, 1, 1],
                                      [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
    def _build_aabb(self):
        buffer = self.step*0.1
        print(self.aabb.shape)
#         tetra = self.tetra
#         nodes = self.nodes
#         ntetra = tetra.shape[0]
#         bb = np.zeros((2,3))
#         step = self.step
#         aabb = self.aabb
#         nx = self.nx
#         ny = self.ny
#         nz = self.nz
#         # create a mask for extracting corners from 
#         corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
#                                           [0, 0, 1, 0, 0, 1, 1, 1],
#                                           [0, 0, 0, 1, 1, 1, 0, 1]]).astype(int)
#         for i in range(ntetra):
#             # copy points into tmp array
#             for k in range(4):
#                 for l in range(3):
#                     points[k,l] = nodes[tetra[i,k],l]
#             for l in range(3):
#                 bb[0,l] = np.min(points[:,l])#,axis=0)
#                 bb[1,l] = np.max(points[:,l])#,axis=0)

#             for j in range(8):
#                 ix = int(bb[corners[0,j],0] // step[0])
#                 iy = int(bb[corners[1,j],1] // step[1])
#                 iz = int(bb[corners[2,j],2] // step[2])
#                 aabb[ix+nx*iy+nx*ny*iz,i] = i

#         return
        build_aabb(self.nx,self.ny,self.nz,self.tetra,self.nodes-self.origin-buffer,self.aabb,self.step+buffer)
        # now sort
    def get_tetra(self,pos):
        pos = np.array(pos)
        tetras = np.zeros(pos.shape[0]).astype(int)
        bcs = np.zeros((pos.shape[0],4))
        for i in range(pos.shape[0]):
            aabbidx = self.pos_to_aabb_index(pos)
            cells = self.aabb[aabbidx[i],:]
            cells=cells[cells>=0]
            npts = len(cells)
            points = self.nodes[self.tetra[cells]]
            vap = pos[i,:] - points[:, 0, :]
            vbp = pos[i,:] - points[:, 1, :]
            # vcp = p - points[:, 2, :]
            # vdp = p - points[:, 3, :]
            vab = points[:, 1, :] - points[:, 0, :]
            vac = points[:, 2, :] - points[:, 0, :]
            vad = points[:, 3, :] - points[:, 0, :]
            vbc = points[:, 2, :] - points[:, 1, :]
            vbd = points[:, 3, :] - points[:, 1, :]

            va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
            vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
            vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
            v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
            
            c = np.zeros((4, npts))
            c[0, :] = va / v
            c[1, :] = vb / v
            c[2, :] = vc / v
            c[3, :] = vd / v
            tetras[i] = cells[np.all(c>0,axis=0)]
            #             print(c)
            bcs[i,:] = c[:,np.all(c>0,axis=0)].T
        return tetras,bcs
#         return self.aabb[aabbidx,:]
    def pos_to_aabb_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step[None, 0]
        iy = iy // self.step[None, 1]
        iz = iz // self.step[None, 2]
        return ix.astype(int)+iy.astype(int)*self.nx+iz.astype(int)*self.nx*self.ny
    
    def aabb_index_to_pos(self,global_index):
        global_index = np.array(global_index)
        x_index = global_index % self.nx
        y_index = global_index // self.nx % \
                  self.ny
        z_index = global_index // self.nx // \
                  self.ny
        xcorner = np.array([0, 1, 0, 0, 1, 0, 1, 1])
        ycorner = np.array([0, 0, 1, 0, 0, 1, 1, 1])
        zcorner = np.array([0, 0, 0, 1, 1, 1, 0, 1])
        xcorners = x_index[:, None] + xcorner[None, :]
        ycorners = y_index[:, None] + ycorner[None, :]
        zcorners = z_index[:, None] + zcorner[None, :]
        corner_pos = self.origin[:,None]+self.step[:,None]*np.vstack([xcorners,ycorners,zcorners])
        print(corner_pos)

In [79]:
def build(origin,maximum,n_tetra,nx,ny,nz):
    minx = origin[0]
    miny = origin[1]
    minz = origin[2]
    maxx = maximum[0]
    maxy =  maximum[1]
    maxz =  maximum[2]
    points = np.array([
        (minx, miny, minz),
        (maxx, miny, minz),
        (maxx, maxy, minz),
        (minx, maxy, minz),
        (minx, miny, maxz),
        (maxx, miny, maxz),
        (maxx, maxy, maxz),
        (minx, maxy, maxz)
    ])
#         self.surfaces['top'] = (
#             (minx, miny, maxz), (maxx, miny, maxz), (maxx, maxy, maxz),
#             (minx, maxy, maxz))
#         self.surfaces['bottom'] = (
#             (minx, miny, minz), (maxx, miny, minz), (maxx, maxy, minz),
#             (minx, maxy, minz))

#         self.points = points
#         # calculate the 3 principal components to find the local coordinate
#         # system
#         self.pca.fit(points)
#         # project the points into this coordinate system
#         newp = self.pca.transform(points)

    lengthU = maxx - minx
    lengthV = maxy - miny
    lengthW = maxz - minz
    boxVol = lengthU * lengthW * lengthV
    correction_factor = 1.91
    maxvol = correction_factor * boxVol / n_tetra
    facets = [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
        [0, 4, 5, 1],
        [1, 5, 6, 2],
        [2, 6, 7, 3],
        [3, 7, 4, 0]
    ]

    # create the mesh
    info = meshpy.tet.MeshInfo()
    # use the projected points to build the mesh
    info.set_points(points)
    info.set_facets(facets)
    meshpy_mesh = meshpy.tet.build(info, max_volume=maxvol,
                                   options=meshpy.tet.Options('pqn'))
    nodes = np.array(meshpy_mesh.points)
    elements=  np.array(meshpy_mesh.elements, dtype=np.int64)
    neighbours = np.array(meshpy_mesh.neighbors, dtype=np.int64)
    return TetMesh2(origin,maximum,nodes,elements,neighbours,nx,ny,nz)#    def __init__(self,origin,maximum,nodes,tetra,neighbours,nx=10,ny=10,nz=10):


In [81]:
for i,t in enumerate([100,1000,5000,7000,10000,15000,20000]):#range(1,10):
    i=i+2
    print(i*i*i)
    mesh = build([0,0,0],[10,10,2],5000,i,i,1)
    for t in mesh.tetra:
        points = mesh.nodes[t]
        try:
            mesh.get_tetra([np.mean(points,axis=0)])
        except:
            print('exception ',i,np.mean(points,axis=0))
#mesh.get_tetra(np.array([[0.69,0.5,.99]]))#np.random.random((100,3)))
# for i in range(100):
#     print(A[i,A[i,:]>0])

8
(4, 6262)


/home/lgrose/python_env/LoopStructural/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


exception  2 [7.17980829 0.46247818 1.08277623]
exception  2 [4.39873833 0.38376786 1.21602251]
exception  2 [0.078125   0.20548654 0.18654567]
exception  2 [0.096832   3.83118853 0.17077637]
exception  2 [3.64939397 0.38404139 1.05036463]
exception  2 [7.05028166 0.12543015 1.8508728 ]
exception  2 [0.078125 9.921875 1.9375  ]
exception  2 [2.48660406 0.25713759 0.18200503]
exception  2 [5.22513421 0.53608469 1.24552198]
exception  2 [0.66071477 0.16342207 0.81284889]
exception  2 [0.25864258 8.41355293 0.12226247]
exception  2 [0.74029958 9.75606789 0.38597143]
exception  2 [0.86969935 0.09335629 0.11817282]
exception  2 [5.20329686 0.15206064 1.84379151]
exception  2 [7.8012587  0.48879882 1.10855483]
exception  2 [0.078125 0.078125 1.9375  ]
exception  2 [0.63992484 9.89292615 1.75372363]
exception  2 [0.12159662 2.0686784  1.59533323]
exception  2 [1.07691766 0.33607585 1.3042107 ]
exception  2 [0.21635641 7.03671202 0.89668481]
exception  2 [0.22369191 0.078125   1.79069761]
exce

exception  2 [2.14693732 0.33068119 0.49513122]
exception  2 [0.15469245 9.37771817 0.84015852]
exception  2 [3.24754993 0.10902578 1.90024536]
exception  2 [0.35621888 6.03210986 0.74928487]
exception  2 [0.60454375 9.85063973 0.72538134]
exception  2 [8.63907563 0.46733314 0.29414402]
exception  2 [0.10728544 9.74228277 0.82109427]
exception  2 [0.3235806  0.65611793 0.96476801]
exception  2 [9.89391935 0.14721228 0.44074544]
exception  2 [2.2025058  0.30295022 1.55907393]
exception  2 [0.68149466 9.66034003 0.89603916]
exception  2 [3.15574157 0.37121756 0.73831206]
exception  2 [0.53115767 8.86910643 0.13350921]
exception  2 [3.61352046 0.11108055 0.12759619]
exception  2 [0.39394383 4.15698251 1.30634746]
exception  2 [0.41019393 0.29078361 0.68689456]
exception  2 [0.40432527 3.62760775 0.70623152]
exception  2 [0.45950174 9.69988692 1.12885063]
exception  2 [0.48983806 8.95876603 1.50776298]
exception  2 [5.10330561 0.21135492 0.85881875]
exception  2 [0.50115728 3.61114783 0.45

exception  2 [0.51619409 1.08496436 1.53526642]
exception  2 [0.1807101  3.61159775 1.27749308]
exception  2 [0.49184192 9.1553831  0.86589163]
exception  2 [0.25229612 7.45979489 1.8854174 ]
exception  2 [8.01192121 0.30238562 0.31347281]
exception  2 [8.27554267 0.17673369 0.27847096]
exception  2 [0.17846847 4.1087972  0.7357135 ]
exception  2 [0.49665124 4.94335925 0.50423611]
exception  2 [0.21288052 6.70369534 1.8508728 ]
exception  2 [0.13625777 6.98792378 0.15620849]
exception  2 [0.13625777 6.80118892 0.27485114]
exception  2 [0.13625777 7.10511128 0.3151594 ]
exception  2 [0.13625777 6.71507955 0.11864265]
exception  2 [0.10483398 8.53074043 0.12226247]
exception  2 [0.22006652 9.24957906 1.89114344]
exception  2 [0.72595294 0.16015625 1.73502478]
exception  2 [7.88631961 0.12191239 1.8854174 ]
exception  2 [0.41579275 5.54053069 0.26235184]
exception  2 [0.18519885 9.81480115 1.9375    ]
exception  2 [7.18451503 0.23277724 1.8508728 ]
exception  2 [3.24499539 0.48229811 0.48

exception  2 [0.17253294 8.5600926  1.11512557]
exception  2 [3.47974635 0.22403658 0.70944309]
exception  2 [3.33796924 0.22403658 0.83946973]
exception  2 [0.35700864 5.7354312  1.01111302]
exception  2 [8.98531556 0.47110582 1.57699663]
exception  2 [0.09946933 7.68402496 1.89478771]
exception  2 [0.23151929 7.83183997 1.89478771]
exception  2 [0.13204995 7.93849096 1.89478771]
exception  2 [0.4329443  0.4409136  0.99849697]
exception  2 [0.43670988 1.97023455 1.86685163]
exception  2 [0.63888983 0.95366279 0.33689612]
exception  2 [8.13653908 0.4681582  1.68376893]
exception  2 [0.36051295 7.06765478 1.09507338]
exception  2 [8.19899233 0.34698237 0.99512163]
exception  2 [8.9085639  0.13675267 0.10268251]
exception  2 [9.07445358 0.28336299 0.10268251]
exception  2 [9.21140973 0.14661033 0.10268251]
exception  2 [1.71869885 0.35163716 1.03224805]
exception  2 [0.35917857 3.84338039 1.00412611]
exception  2 [8.64836513 0.46768325 1.72692973]
exception  2 [0.42201587 2.73213617 1.22

exception  2 [4.35695952 0.43677203 1.68184464]
exception  2 [0.3113863  9.35367892 1.57524632]
exception  2 [0.20544955 8.87773005 0.10427413]
exception  2 [0.30552431 0.92581316 1.52662208]
exception  2 [0.70885383 4.60718738 0.20733968]
exception  2 [0.40699528 2.43255907 0.28945512]
exception  2 [0.41918869 7.49040107 1.71730745]
exception  2 [4.01609708 0.38517981 1.21176272]
exception  2 [0.41177491 5.88447557 1.68736614]
exception  2 [0.12159662 1.9514909  1.74468626]
exception  2 [0.16831728 1.00900819 1.12734267]
exception  2 [1.35505645 0.10700031 1.72399026]
exception  2 [8.51283815 0.32444194 0.70140834]
exception  2 [8.23064531 0.17638721 1.18873011]
exception  2 [0.30035539 1.3402462  0.41135966]
exception  2 [6.90535365 0.29757985 1.55494518]
exception  2 [0.60272598 3.15266161 1.14478864]
exception  2 [2.83998184 0.27731673 1.85064697]
exception  2 [2.60460991 0.16829095 1.66975082]
exception  2 [2.69079971 0.16829095 1.85064697]
exception  2 [1.5934558  0.68156099 0.33

exception  2 [5.61410803 0.55663453 0.17030761]
exception  2 [5.49999996 0.16203562 0.79826464]
exception  2 [5.89407573 0.16203562 0.66582893]
exception  2 [5.98084827 0.16203562 0.79628515]
exception  2 [1.36724991 0.54747327 0.96427561]
exception  2 [0.97355737 0.39564273 0.42497077]
exception  2 [0.68346513 0.33033567 0.99637485]
exception  2 [1.12293374 0.35826949 0.25892428]
exception  2 [0.24920851 6.67735779 1.58682721]
exception  2 [9.0266195  0.32292407 1.34837743]
exception  2 [0.15699581 6.56839889 0.68928195]
exception  2 [0.69458292 6.07872679 0.35511885]
exception  2 [1.62893593 0.16916226 1.18421825]
exception  2 [1.78258704 0.16916226 1.31128494]
exception  2 [1.80750475 0.56345402 1.84205516]
exception  2 [1.38510321 0.16916226 1.31604542]
exception  2 [1.25645738 0.16916226 1.22388234]
exception  2 [0.36774057 6.66957178 1.26729853]
exception  2 [7.25085366 0.30437133 0.30203851]
exception  2 [0.29894253 7.89619379 1.59944189]
exception  2 [0.16689257 7.62715664 1.26

IndexError: Out of bounds on buffer access (axis 0)